# Prototype of lipreading pipeline

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

import random
import math
import time

# model file, encoder, decoder and seqtoseq
from model_temp import *
# utils file
from utils import *
# Get landmark using vocadataset.py
from data.vocaset import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

In [2]:
# Get landmark from vocadaset class
#trainset = vocadataset("train", landmark=True)
trainset = vocadataset("train", landmark=True, mouthOnly=True)
#landmark, labels = trainset[0]

trainloader = DataLoader(trainset, batch_size=20, collate_fn=collate_fn, shuffle=True)

#landmarks, len_landmark, label, len_label = next(iter(trainloader))


In [3]:
vocabulary = create_vocabulary(blank="@")

In [ ]:
# Create a mapping from characters to indices
char_to_index = {char: index for index, char in enumerate(vocabulary)}

In [8]:
# Convert the sequence and target to indices
#sequence_indices = [char_to_index[char] for char in sequence]

label_t = char_to_index_batch(label, vocabulary)

#target_indices = [char_to_index[char] for char in labels]
#target_tensor = torch.tensor(target_indices)

In [7]:
INPUT_DIM = 36*3
INPUT_DIM = 36*3
HID_DIM = 64
output_dim = len(vocabulary)

model = only_Decoder2(INPUT_DIM, HID_DIM, 2, len(vocabulary)).to(device)



In [ ]:
# Define the CTC loss function
ctc_loss = nn.CTCLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10000
for epoch in range(num_epochs):
    
    for i in range(landmarks.shape[0]):
        landmarks[i] = landmarks[i].to(device) 
        label_t[i] = label_t[i].to(device)
        optimizer.zero_grad()
        target_tensor = label_t[i]

        

        reshaped_landmark = torch.reshape(landmarks[i], (landmarks[i].shape[0], landmarks[i].shape[1]*landmarks[i].shape[2]))
        reshaped_landmark = reshaped_landmark.to(device)
        target_tensor = target_tensor.to(device)
        len_landmark[i] = len_landmark[i].to(device) 
        output = model(reshaped_landmark[None, : , :], len_landmark[i][None])
        output = output.permute(1,0,2)
        input_lengths = torch.full((1,), output.size(0), dtype=torch.long)
        target_lengths = torch.full((target_tensor.size(0),), target_tensor.size(0), dtype=torch.int32)
        
        loss = ctc_loss(torch.nn.functional.log_softmax(output, dim=2), target_tensor, input_lengths, target_lengths[0])
        loss.backward()
        optimizer.step()

        e = torch.argmax(output, dim=2).squeeze(1)
        output_sequence = ''.join([vocabulary[index] for index in e])
        #print(output_sequence)
        if(epoch + 1) % 100 == 0:
            f = open("prova_.txt", "a")
            f.write(label[i]+"\n")
            f.write(output_sequence+"\n")
            f.close() 
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")
        #e = torch.argmax(output, dim=2).squeeze(1)
        #output_sequence = ''.join([vocabulary[index] for index in e])
        #print(output_sequence)



In [ ]:
# Define the CTC loss function
ctc_loss = nn.CTCLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 500
for landmarks, len_landmark, label, len_label in trainloader:
    #landmarks, len_landmark, label, len_label = next(iter(trainloader))
    label_t = char_to_index_batch(label, vocabulary)
    
    for epoch in range(num_epochs):
        
        for i in range(landmarks.shape[0]):
            landmarks[i] = landmarks[i].to(device) 
            label_t[i] = label_t[i].to(device)
            optimizer.zero_grad()


            #remove padding from dataloader
            nw = landmarks[i][:len_landmark[i], :, :].clone().detach().requires_grad_(True)
            nw_l = label_t[i][:len_label[i]].clone().detach()
            
            target_tensor = nw_l
            reshaped_landmark = torch.reshape(nw, (nw.shape[0], nw.shape[1]*nw.shape[2]))
            reshaped_landmark = reshaped_landmark.to(device)
            target_tensor = target_tensor.to(device)
            len_landmark[i] = len_landmark[i].to(device) 
            output = model(reshaped_landmark[None, : , :], len_landmark[i][None])
            output = output.permute(1,0,2)
            input_lengths = torch.full((1,), output.size(0), dtype=torch.long)
            target_lengths = torch.full((target_tensor.size(0),), target_tensor.size(0), dtype=torch.int32)
            
            loss = ctc_loss(torch.nn.functional.log_softmax(output, dim=2), target_tensor, input_lengths, target_lengths[0])
            loss.backward()
            optimizer.step()

            e = torch.argmax(output, dim=2).squeeze(1)
            output_sequence = ''.join([vocabulary[index] for index in e])
            #print(output_sequence)
            """if(epoch + 1) % 100 == 0:
                f = open("prova_.txt", "a")
                f.write(str(nw_l.tolist())+"\n")
                f.write(output_sequence+"\n")
                f.close() 
        """
        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")
            #e = torch.argmax(output, dim=2).squeeze(1)
            #output_sequence = ''.join([vocabulary[index] for index in e])
            #print(output_sequence)



In [ ]:
# Define the CTC loss function
ctc_loss = nn.CTCLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)


###########

#trainset = vocadataset("train", landmark=True)
trainset_ = vocadataset("train", landmark=True, mouthOnly=True)
#landmark, labels = trainset[0]
trainset_ = trainset_.to(device)
trainloader_ = DataLoader(trainset_, batch_size=1, collate_fn=collate_fn, num_workers=8)


###########à

In [ ]:
INPUT_DIM = 36*3
INPUT_DIM = 36*3
HID_DIM = 64
output_dim = len(vocabulary)

model = only_Decoder2(INPUT_DIM, HID_DIM, 2, len(vocabulary)).to(device)

In [ ]:
trainset_[0][0].shape

In [ ]:
torch.tensor(trainset_[0][0].shape[0]).shape

In [ ]:
# Define the CTC loss function
ctc_loss = nn.CTCLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)


###########
trainset_ = vocadataset("train", landmark=True, mouthOnly=True)


# Training loop
num_epochs = 10000
for epoch in range(num_epochs):

    for i  in range(10):
        optimizer.zero_grad()

        landmarks = trainset_[i][0].to(device) 
        
        label_t = char_to_index_batch(trainset_[i][1], vocabulary)
        label_t = label_t.to(device)
        target_tensor = label_t
        reshaped_landmark = torch.reshape(landmarks, (landmarks.shape[0], landmarks.shape[1]*landmarks.shape[2]))
        reshaped_landmark = reshaped_landmark.to(device)
        target_tensor = target_tensor.to(device)
        len_landmark = torch.tensor(landmarks.shape[0]).to(device) 
        output = model(reshaped_landmark[None, : , :], len_landmark[None])
        output = output.permute(1,0,2)
        input_lengths = torch.full((1,), output.size(0), dtype=torch.long)
        target_lengths = torch.full((target_tensor.size(0),), target_tensor.size(0), dtype=torch.int32)
        
        loss = ctc_loss(torch.nn.functional.log_softmax(output, dim=2), target_tensor.squeeze(), input_lengths, target_lengths[0][None])
        loss.backward()
        optimizer.step()

        e = torch.argmax(output, dim=2).squeeze(1)
        output_sequence = ''.join([vocabulary[index] for index in e])
        #print(output_sequence)
        if(epoch + 1) % 100 == 0:
            f = open("prova_.txt", "a")
            f.write(label[i]+"\n")
            f.write(output_sequence+"\n")
            f.close() 
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")
        #e = torch.argmax(output, dim=2).squeeze(1)
        #output_sequence = ''.join([vocabulary[index] for index in e])
        #print(output_sequence)



In [4]:

valset = vocadataset("val", landmark=True, mouthOnly=True)
#landmark, labels = trainset[0]

valloader = DataLoader(valset, batch_size=1, collate_fn=collate_fn)


In [5]:
def process_string(input_string, blank= "@"):
    output_string = ""
    current_char = ""

    for char in input_string:
        if char != current_char:
            if char == blank:
                output_string += ''
            else:
                output_string += char
                
            current_char = char
        else:
            output_string += ''


    return output_string.strip()


In [8]:


model.load_state_dict(torch.load("/home/prasanna/Documents/UNIFI/Computer Graphics/LipReading/lipreading/m/model20sent_4.pt"))
model.eval()

real_sentences = []
pred_sentences = []

with torch.no_grad():

    for landmarks, len_landmark, label, len_label in valloader:

        # reshape the batch from [batch_size, frame_size, num_landmark, 3] to [batch_size, frame_size, num_landmark * 3] 
        landmarks = torch.reshape(landmarks, (landmarks.shape[0], landmarks.shape[1], landmarks.shape[2]*landmarks.shape[3]))
        
        #variable to recover later the target sequences
        label_list = label

        # label char to index
        label = char_to_index_batch(label, vocabulary)

        # move data to GPU!
        landmarks = landmarks.to(device)
        len_landmark = len_landmark.to(device)
        label = label.to(device)
        len_label = len_label.to(device)

        output = model(landmarks, len_landmark)
        output = output.permute(1, 0, 2)
        # scrittura nel file del outuput e della frase originale

        real_sentences, pred_sentences = write_results(len_label, label_list, output.detach(), valloader.batch_size, vocabulary, real_sentences, pred_sentences)

    pred_sentences = list(map(lambda x:process_string(x),pred_sentences))
    save_results(f"./results/validation.txt", real_sentences, pred_sentences, overwrite=True)


In [17]:
import nltk
nltk.download('punkt')
def calculate_bleu(model_sentences, reference_sentences):
    # Tokenize the sentences
    model_sentences = [nltk.word_tokenize(sentence) for sentence in model_sentences]
    reference_sentences = [[nltk.word_tokenize(sentence)] for sentence in reference_sentences]

    # Compute BLEU score
    smoothie = nltk.translate.bleu_score.SmoothingFunction()
    bleu_score = nltk.translate.bleu_score.corpus_bleu(reference_sentences, model_sentences, smoothing_function=smoothie.method1)

    return bleu_score

[nltk_data] Downloading package punkt to /home/prasanna/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [13]:
pred_sentences

['tcideeaetved ere doneddhim oshoraf.',
 'aisos s acuieaes fasd dsaaintinlg ig hsadid mad.',
 'eles md fews tlrcreiaeedsasteiveed fr re emoa acolyilr oc m ae tvn rere efre forlee dm mfoformaorcdcvg.',
 'hiadr oanniitiniit aeac woir iutol oolom jasainct?',
 'e tebacnthndantdau s wm wshcranin ird to orraetihsoi ied aiu adrr.',
 'ntentifertarergtegeegaegeteittrestitntttnihtniwtititnan.',
 't th ce idteaei aaisdeatee metiietsouu.',
 'taiturcgtsrhvi fis oi str faratisto onraenivin itctrot.',
 'hasirohtegdeits tocrinolg coiovnenvgveinl aegegildi  chaotedeaettdtcc.',
 'd manai oem o on nsrgngbakh cehci wctliescoart imoghioft?',
 'tanuotsdechi ietcedstueitcthihi odm sarhrdstrs ihsfocth tt.',
 'intts rtntaiemtsat rtrairnianiaoarti foc oooooofsth.',
 'actngstc ththe ot scvschchidtkog fmcle wal ?',
 'tnetgoirtsr aauocsesoalecororicsfss.',
 'rsomistri raed fed ussasnnsietivetcd.',
 'hrtve oeatyisgiseacgih ethn arotactritataes st avothe g?',
 'ttowm hein hae tasdsd esd snicuieaheaestestvi trcm aeid

In [25]:
calculate_bleu(['Ciao mi chiamo prasanna'],['Ciao mi chiamo prasanna'])

1.0

In [ ]:
torch.save(model.state_dict(), "models/model20sent_4.pt")